# Introduction
Tutorial on using the functions in mofstructures. The tutorial covers all the basic functionalities of mofstructures. The module can be applied to any crystalline porous system such as zeolites, COFs and MOFs. The module has the following functionalities 

1. Computation of geometry properties of porous systems. It calls zeo++ in the background and enables a quick computation of all porosity information and returns a python dictionary object.

2. Automated removal of unbound guest molecules

3. Deconstruction of metal organic frameworks into building units. And for each buidling units (organic ligand, metal cluster, organic sbu and metal sbu) computes their cheminformatic identifiers such as SMILES strings, inchi and inchikey 

4. wraps systems around unit cell so as to remove effect of pbc. 

5. Seperation of building units into regions 

In [ ]:
from ase.io import read, write # Reading and writing into different formats

In [2]:
import mofstructure.mofdeconstructor as MOF_deconstructor # Manipulating MOFs

In [ ]:
from mofstructure.porosity import zeo_calculation # For compution porosity

In [ ]:
from IPython.display import HTML # For visualization 

## Basics
Read a file. Any ase.io file type can be read

In [ ]:
structure = read('data/Kick_0.cif')

In [6]:
print (structure)

Atoms(symbols='C158H93Cl2N13O34Zr6', pbc=True, cell=[19.41, 19.41, 19.41], spacegroup_kinds=...)


We now write a simple python function that can convert the ase atom object to an html object to enable visualisation of the structures. This is not part of the mofstructure module, it is here mainly for visualisation

In [ ]:
def atoms_to_html(atoms):
    'Return the html representation the atoms object as string'

    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+', suffix='.html') as ntf:
        atoms.write(ntf.name, format='html')
        ntf.seek(0)
        html = ntf.read()
    return html

 For visualizing the structure. You can also decide to use
1. Mercury 
2. Vesta
3. gview
4. Avogadro e.t.c

To visualize the structure we just load
1. convert it to html
2. run HTML

In [8]:
struct_2_html = atoms_to_html(structure)

In [ ]:
HTML(struct_2_html)

On inspection of the structure, we realise that there is an unbound guest molecule at the center of the pore. Ideally, we will want to remove this guest molecule before computing the porosity of the system and deconstructing the system. 

To enable the guest removal we call the *MOF_deconstructor.remove_unbound_guest* function. This returns indices of the guest free structure, which we can then pass to the ase atom object

In [10]:
# Indices of guest free system

In [ ]:
guest_free_indices = MOF_deconstructor.remove_unbound_guest(structure)

In [ ]:
# guest free ase atom object

In [ ]:
guest_free_atoms = structure[guest_free_indices]

Now we can visualise the system to see that all see that the guest has been remove by calling our html visualiser 

In [ ]:
HTML(atoms_to_html(guest_free_atoms))

Now that we have remove the guest molecule we can now proceed to computing the porosity of our system and also deconstructing the system into its various building units and extracting some informatic identifiers for the system.

In [15]:
# Computation of porosity

# Computation of porosity
To compute the porosity, the modules call zeo++ in the backround. This is a python extension of zeo++ that we implemented to run smoothly with any ase atom object.  We can extent it to run with other molecule objects. Email me if you need any further information or adaptation. 

In [ ]:
porosity = zeo_calculation(guest_free_atoms, probe_radius=1.86, number_of_steps=5000)

(True, None)
Reading input file: tmp.cssr
Box dimensions:
  va=(19.410000 0 0)
  vb=(0.000000 19.410000 0)
  vc=(0.000000 0.000000 19.410000)

Total particles = 276

Internal grid size = (4 4 4)

Using voro++ with radii for particles.
Performing Voronoi decomposition.
Volume check:
  Total domain volume  = 7312.680621
  Total Voronoi volume = 7312.680621
Voronoi decomposition finished. Rerouting Voronoi network information.
Finished rerouting information.
Voronoi network with 1949 nodes. 1040 of them are accessible. 

Finding channels and pockets in Dijkstra network of 1949 node(s). 1040 are expected to compose pores.
Analyzed and assigned 1949 nodes.
Identified 1 channels and 0 pockets.
1040 nodes assigned to pores. 
Box dimensions:
  va=(19.410000 0 0)
  vb=(0.000000 19.410000 0)
  vc=(0.000000 0.000000 19.410000)

Total particles = 276

Internal grid size = (4 4 4)

Using voro++ with radii for particles.
Performing Voronoi decomposition.
Volume check:
  Total domain volume  = 7312.6

In [17]:
print (porosity)

{'AV_Volume_fraction': 0.249, 'AV': 1820.86, 'ASA': 1397.71, 'Number_of_channels': 1, 'LCD': 16.00014, 'lfpd': 16.00014, 'PLD': 4.60266}


Here we use a probe radius of 1.86 A, which is equivalent to the covalent radius of N2. You can change the radius to any value. We also use a default of 5000 monte carlo simulation steps to compute the accessible surface area (ASA) and accessible volume (AV). 
The following data and units are returned 
1. Accessible volume fraction
2. Accessible volume (AV), units = A^3
3. Accessible surface area (ASA), units = A^2
4. Largest cavity diameter (LCD), units = A
5. Free sphere (lfpd), units = A
6. largest included sphere along free sphere path (LFPD)
7. Pore limiting diameter(PLD), units = A

In [18]:
# convert to a pandas dataframe

In [19]:
import pandas as pd

In [ ]:
df = pd.DataFrame(porosity, index=[0])

In [ ]:
print (df)

   AV_Volume_fraction       AV      ASA  Number_of_channels       LCD   
0               0.249  1820.86  1397.71                   1  16.00014  \

       lfpd      PLD  
0  16.00014  4.60266  


In [22]:
# save to csv 

In [ ]:
df.to_csv('test/porosity_data.csv')

# MOF deconstruction
We provide two approaches to deconstruct MOFs. The first approach decontsructs the MOFs into the secondary building units while the second approach deconstructs it into organic ligands and metal clusters. 

## Deconstruction into secondary building units
The secondary building units are group of molecules, nodes and linkers, that can be assembled in various topologies to create new hypothetical MOFs. These are the building units that are used by AuToGraFS (https://github.com/DCoupry/autografs) for buidling hypothetical MOFs.

In [ ]:
 connected_components, atoms_indices_at_breaking_point, porpyrin_checker, all_regions = MOF_deconstructor.\
        secondary_building_units(guest_free_atoms)

In [ ]:
metal_sbus, organic_sbus, building_unit_regions = MOF_deconstructor.\
        find_unique_building_units(
            connected_components,
            atoms_indices_at_breaking_point,
            guest_free_atoms, porpyrin_checker,
            all_regions,
            cheminfo=True
        )

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zr(6); O(1); Zr(5); C(2); C(3); Metal was disconnected; Proton(s) added/removed
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zr(6); O(1); Zr(5); C(2); C(3); Metal was disconnected; Proton(s) added/removed
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)


### Secondary building units
The secondary building units are sorted in a list. The number of element in each list corresponds to the number of unique building units found in the MOF

#### Metal secondary building units
The metal secondary building unit is encorded with a number of information coded in each unique building unit. These information can be obtained using the ase_atom.info the most important information are:
1. Type of secondary building unit
2. Smile strings
3. Number of point of extensions
4. inchikey
5. inchi
6. Atom indices mapping 

In [26]:
for i, sbu_metal in enumerate(metal_sbus):
    print ('1. Type of sbu:', sbu_metal.info['sbu_type'])
    print('\n')
    print ('2. Smile strings:', sbu_metal.info['smi'])
    print('\n')
    print ('3. Number_of_point_of_extension:', len(sbu_metal.info['point_of_extension']))
    print('\n')
    print ('4. inchikey:', sbu_metal.info['inchikey'])
    print('\n')
    print ('5. inchi:', sbu_metal.info['inchi'])
    print('\n')
    print ('6. Atom indices mapping:', sbu_metal.info['atom_indices_mapping'])
    print('\n')

1. Type of sbu: UIO66_sbu


2. Smile strings: O1[Zr]234(O[C]O[Zr]567(O[C]O[Zr]8(O[C]O[Zr]9(O[C]O[Zr]1(O[C]O8)(O[C]O3)O[C]=O)(O[C]O6)O[C]O[Zr](O)(O[C]O7)(O9)O[C]O4)(O[C]=O)O5)O2)O.[OH].[OH]


3. Number_of_point_of_extension: 12


4. inchikey: CSIPMCQAIJNSNV-UHFFFAOYSA-L


5. inchi: 1S/12CO2.2H2O.2HO.4O.6Zr/c12*2-1-3;;;;;;;;;;;;;;/h;;;;;;;;;;;;2*1H2;2*1H;;;;;;;;;;/q10*-2;2*-1;;;;;;;;;6*+4/p-2


6. Atom indices mapping: [[0, 1, 2, 3, 4, 5, 6, 7, 159, 70, 63, 62, 60, 61, 64, 65, 66, 67, 68, 69, 117, 115, 116, 131, 129, 130, 161, 175, 176, 177, 178, 179, 71, 72, 164, 163, 162, 239, 238, 240, 260, 73, 74, 158, 160, 44, 42, 43, 216, 215, 217, 144, 143, 145]]




#### Visualise the sbu 
The type of metal sbu is shown to be the Zr UIO66 as shown below


In [ ]:
ase_atom = metal_sbus[0]

In [ ]:
HTML(atoms_to_html(ase_atom))

#### Organic secondary building units
The organic secondary building unit is encorded with a number of information coded in each unique building unit. These information can be obtained using the ase_atom.info the most important information are:

1. Smile strings
2. Number of point of extensions
3. inchikey
4. inchi
5. Atom indices mapping

The organic sbu can be form save or written by simply. organic_sbus[0].write('organic_sbu_1.xyz)

In [ ]:
for i, sbu_organic in enumerate(organic_sbus):
    print('\n')
    print ('1. Smile strings:', sbu_organic.info['smi'])
    print('\n')
    print ('2. Number_of_point_of_extension:', len(sbu_organic.info['point_of_extension']))
    print('\n')
    print ('3. inchikey:', sbu_organic.info['inchikey'])
    print('\n')
    print ('4. inchi:', sbu_organic.info['inchi'])
    print('\n')
    print ('5. Atom indices mapping:', sbu_organic.info['atom_indices_mapping'])
    print('\n')



1. Smile strings: [C]1=CC=C(C=C1)/C/1=c\2/[nH]/c(=C(\C3=N/C(=C(\c4[nH]c(/C(=C\5/C=CC1=N5)/C1=CC=[C]C=C1)cc4)/C1=CC=[C]C=C1)/C=C3)/C1=CC=[C]C=C1)/cc2


2. Number_of_point_of_extension: 4


3. inchikey: FKAPXTLNAQEVNG-LWQDQPMZSA-N


4. inchi: 1S/C44H26N4/c1-5-13-29(14-6-1)41-33-21-23-35(45-33)42(30-15-7-2-8-16-30)37-25-27-39(47-37)44(32-19-11-4-12-20-32)40-28-26-38(48-40)43(31-17-9-3-10-18-31)36-24-22-34(41)46-36/h5-28,45,48H/b41-33-,41-34-,42-35-,42-37-,43-36-,43-38-,44-39-,44-40-


5. Atom indices mapping: [[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 261, 262, 263, 264, 265, 266, 169, 168, 166, 165, 171, 170, 173, 172, 167, 174, 267, 268, 269, 270, 271, 272, 273, 274, 275], [75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112

In [30]:
HTML(atoms_to_html(organic_sbus[0]))
    

In [31]:
for i, sbu_organic in enumerate(organic_sbus):
    sbu_organic.write('test/organic_sbu_'+str(i)+'.xyz')

### Organic ligand and metal cluster
The organic ligands and metal cluster are sorted in a list. The number of element in each list corresponds to the number of unique organic ligands and metal cluster found in the MOF

In [ ]:
connected_components2, atoms_indices_at_breaking_point2, porpyrin_checker2, all_regions2 = MOF_deconstructor.\
        ligands_and_metal_clusters(guest_free_atoms)
metal_clusters, organic_ligands, building_unit_regions = MOF_deconstructor.\
    find_unique_building_units(
        connected_components2,
        atoms_indices_at_breaking_point2,
        guest_free_atoms,
        porpyrin_checker2,
        all_regions2,
        cheminfo=True
    )

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zr(3); Zr(2); Zr(1); Metal was disconnected; Proton(s) added/removed
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zr(3); Zr(2); Zr(1); Metal was disconnected; Proton(s) added/removed
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)


In [ ]:
HTML(atoms_to_html(metal_clusters[0]))

In [ ]:
HTML(atoms_to_html(organic_ligands[0]))

#### Another example
All building units can be compiled and safed in a directory by simply calling the *workflow* function in the buildingunits module

In [ ]:
from mofstructure import buildingunits

In [ ]:
buildingunits.work_flow('data/EDUSIF.cif', 'test')

(True, None)


Order 4 vertex memory scaled up to 16


Box dimensions:
  va=(19.410000 0 0)
  vb=(0.000000 19.410000 0)
  vc=(0.000000 0.000000 19.410000)

Total particles = 276

Internal grid size = (4 4 4)

Using voro++ with radii for particles.
Performing Voronoi decomposition.
Volume check:
  Total domain volume  = 7312.680621
  Total Voronoi volume = 7312.680621
Voronoi decomposition finished. Rerouting Voronoi network information.
Finished rerouting information.
Reading input file: tmp.cssr
Box dimensions:
  va=(25.832000 0 0)
  vb=(0.000000 25.832000 0)
  vc=(0.000000 0.000000 25.832000)

Total particles = 424

Internal grid size = (4 4 4)

Using voro++ with radii for particles.
Performing Voronoi decomposition.
Volume check:
  Total domain volume  = 17237.492730
  Total Voronoi volume = 17237.492730
Voronoi decomposition finished. Rerouting Voronoi network information.
Finished rerouting information.
Voronoi network with 640 nodes. 384 of them are accessible. 

Finding channels and pockets in Dijkstra network of 640 node(s). 384 ar

Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zn(3); C(2); Zn(4); Metal was disconnected
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zn(3); C(2); Zn(4); Metal was disconnected
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)


sbu IRMOF_sbu
point_of_extension 6


*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zn(1); Metal was disconnected
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zn(1); Metal was disconnected
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)


The building units and porosity have all be safed in a folder call test.

# Miscellaneous
There are lots of other helper functions found in mofstructures. The most common functions are:
1. Extract unique metals and metal coordination numbers
2. Wrap structures to unit cell
3. Separate building units into regions
4. Convert ase to openbabel and rdkit 
   

In [ ]:
# unique metal and coordination number

In [ ]:
metal, metal_coordination = MOF_deconstructor.metal_coordination_number(guest_free_atoms)

In [39]:
print (metal)

['Zr']


In [40]:
print (metal_coordination)

{'Zr': 12}


#### Wrap structure to unit cell
Most times, the presence of minimum image convention makes atoms to look uncoordinated when visualised. One way to solve this is by wrapping the coordinates to the unit cell. Moreover, most people will like convert a cif file to xyz so as to run a cp2k calculation. But with atoms appearing uncoordinated, this makes it sometime difficult. One way of doing that is by simply calling the function *wrap_systems_in_unit_cell*

In [ ]:
# Before wrapping 

In [42]:
HTML(atoms_to_html(guest_free_atoms))

In [43]:
#After wrapping 

In [44]:
wrap_atoms = MOF_deconstructor.wrap_systems_in_unit_cell(guest_free_atoms)

In [ ]:
HTML(atoms_to_html(wrap_atoms))

In [ ]:
mof_5 = read('data/EDUSIF.cif')

In [ ]:
HTML(atoms_to_html(mof_5))

#### Replace point of extention with dummy 
Most often one will like to either filled in the point of extensions with either dummy atoms (X) or with hydrogen atoms to neutralise the building unit before performing any computations. There are several automated ways to fill in hydrogen but the problem with these methods is that they all rely on atom and bondtyping. Meaning that if there are any distortion in geometry, then the bondtyping will be wrong. 

To avoid this problem, we actually store the information about the position and coordinates of neutralising or filling with dummies. This information is store in .info['point_of_extension'].

We will illustrate this using MOF-5. We will begin by deconstructing it into its secondary building units and then filling this building units with dummies. 

In [48]:
# Deconstruct MOF-5 into building units

In [49]:
 connected_components, atoms_indices_at_breaking_point, porpyrin_checker, all_regions = MOF_deconstructor.\
        secondary_building_units(mof_5)
metal_sbu, organic_sbu, building_unit_regions = MOF_deconstructor.\
    find_unique_building_units(
        connected_components,
        atoms_indices_at_breaking_point,
        mof_5,
        porpyrin_checker,
        all_regions,
        wrap_system=False,
        cheminfo=True
        
    )


*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zn(3); C(2); Zn(4); Metal was disconnected
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): Zn(3); C(2); Zn(4); Metal was disconnected
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)


In [50]:
# metal sbus are found in the 'metal_sbu' and the organic sbu are found in organic_sbu

In [51]:
# The number of unique metal sbu present are :

In [52]:
print ('Number of unique metal sbu  ', len(metal_sbu))

Number of unique metal sbu   1


In [53]:
#The number of unique oragnic ligands are 

In [54]:
print ('number of unique organic sbu  ', len(organic_sbu))

number of unique organic sbu   1


We see that in MOF 5, there is one unique metal cluster and one unique organic sbu. These are respectively the Zr cluster and the benzene with the carboxylate cut out to form the Zr cluster. 

In [55]:
# A visual representation of the Zr cluster

In [56]:
HTML(atoms_to_html(metal_sbu[0]))

In [57]:
# A visual representation of the organic sbu

In [58]:
#HTML(atoms_to_html(organic_sbu[0]))

In [59]:
#Now adding dummy to sbus

In [60]:
#For the metal sbu. The indices of atoms at breaking points are 

In [61]:
dummy_indices = metal_sbu[0].info['point_of_extension']

In [62]:
# ASE_atom object for this dummy 

In [63]:
dummy_ase_atom_object = mof_5[dummy_indices]

In [64]:
# Replace symbols with X

In [65]:
for atom in dummy_ase_atom_object:
    atom.symbol = 'H'

In [66]:
# Metal sbu with dummy

In [67]:
metal_sbu_with_dummy = MOF_deconstructor.wrap_systems_in_unit_cell(metal_sbu[0]+dummy_ase_atom_object)

In [68]:
HTML(atoms_to_html(metal_sbu_with_dummy))